# Modelo de detección de tono

## Descargas, instalaciones e importaciones

In [19]:
# Importaciones necesarias

# Para dataframe
import pandas as pd
# Para limpieza de texto
import nltk
from nltk.corpus.reader.tagged import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.text import FreqDist
# Para guardar resultados en archivos
import pickle
# Para vectorizar el texto
from sklearn.feature_extraction.text import TfidfVectorizer
# Para preparar los datos vectorizados
from sklearn.model_selection import train_test_split

In [20]:
# prompt: Importame los mejores modelos para clasificación de texto
from sklearn.ensemble import RandomForestClassifier
# metricas para clasificación
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [ ]:
# Descargas de nltk necesarias

# Signos de puntuación
nltk.download('punkt')
# Stopwrds
nltk.download("stopwords")
# Nombres propios
nltk.download("names")
# Lemmatizer
nltk.download('wordnet')


In [22]:
# Declarando las listas de cosas que limpiar en el texto
stopwords = nltk.corpus.stopwords.words('english')
nombres = nltk.corpus.names.words()

## Unboxing y preparación de la data

In [23]:
# Creando nuestro dataframe
df = pd.read_csv("go_emotions_dataset.csv")

In [ ]:
df.info()

In [ ]:
df.shape

In [ ]:
# Viendo cantidad de entradas por clasificación
columns = df.columns.to_list()
columns

for column in columns[3:]:
  print(f"""
    {column}
    {(df[column] != 0).sum()}
  """)

In [ ]:
# Ver la estructura del dataframe
df.head(3)

In [28]:
# Quitando el one hot encoding
df = pd.concat([df.iloc[:, :3], df.iloc[:, 3:].idxmax(axis=1)], axis=1, )

In [29]:
df = df.rename(columns={0: "emotions"})

In [30]:
# Dropeando columnas innecesarias
df = df.drop(["id", "example_very_unclear"], axis= 1)

In [ ]:
df.head()

In [32]:
# Quitar todas las entradas neutrales (Confunden el modelo)

df = df[df["emotions"] != "neutral"]

In [33]:
df.loc[(df['emotions'] == 'curiosity') | (df['emotions'] == 'confusion'), 'emotions'] = 'perplexity'

In [34]:
df.loc[(df['emotions'] == 'anger') | (df['emotions'] == 'annoyance'), 'emotions'] = 'irritation'

In [35]:
df.loc[(df['emotions'] == 'admiration'), 'emotions'] = 'appreciation'
df.loc[(df['emotions'] == 'approval'), 'emotions'] = 'appreciation'
df.loc[(df['emotions'] == 'love'), 'emotions'] = 'appreciation'

In [36]:
df.loc[(df['emotions'] == 'amusement'), 'emotions'] = 'joy'

In [37]:
df.loc[(df['emotions'] == 'remorse'), 'emotions'] = 'sadness'

In [38]:
df.loc[(df['emotions'] == 'grief'), 'emotions'] = 'sadness'

In [39]:
df.loc[(df['emotions'] == 'excitement'), 'emotions'] = 'joy'

In [40]:
df.loc[(df['emotions'] == 'caring') | (df['emotions'] == 'optimism'), 'emotions'] = 'compassion'

In [ ]:
df.head(100)

In [ ]:
# Visualización de las emociones únicas

print("Emociones únicas:", df['emotions'].unique())

## Trtamiento de la texto para su uso

In [43]:
# Instanciando el lematizador
lematizador = WordNetLemmatizer()

In [44]:
# Palabras indeseadas
palabras_indeseadas = ['he', 'a', 'the', 'in', 'an', 'it', 'she', 'ca', 'wo']

In [45]:
# Función para tokenizar y limpiar el texto
def obtener_tokens(text: str) -> list:

  """
  Función para tokenizar y limpiar el texto.
  Se obvian los tokens que sean stopwords,
  nombres, números, caracteres especiales,
  letras palabras indeseadas. Los demás tokens
  se lematizan y son recolectados.
  """

  tokens_crudos = word_tokenize(text)
  tokens = []
  for token in tokens_crudos:
    if token in stopwords: continue
    if token in nombres: continue
    if not token.isalpha():  continue
    if (len(token) < 2): continue
    token = token.lower()
    if token in palabras_indeseadas: continue
    token = lematizador.lemmatize(token)
    tokens.append(token)

  return tokens



In [46]:
# Crear los vocabularios
vocabulario = {emotion:[] for emotion in df['emotions'].unique()}
vocabulario_completo = []
for _, row in df.iterrows():
  tokens = obtener_tokens(row["text"])
  vocabulario[row['emotions']] += tokens
  vocabulario_completo += tokens

In [ ]:
# Ver los tokens mas comunes en vocabulario_completo

# Crear un objeto FreqDist para contar las frecuencias de las palabras
freq_dist = FreqDist(vocabulario_completo)

# Obtener las palabras más comunes
palabras_mas_comunes = freq_dist.most_common(10)

# Imprimir las palabras más comunes
print("Palabras más comunes en vocabulario_completo:")
for palabra, frecuencia in palabras_mas_comunes:
  print(f"{palabra}: {frecuencia}")


In [ ]:
# Contar las ocurrencias de cada token por emoción
vocabulario_por_emocion = {emotion: FreqDist(tokens) for emotion, tokens in vocabulario.items()}

# Obtener los tokens más comunes por emoción
tokens_mas_comunes_por_emocion = {emotion: freq_dist.most_common(15) for emotion, freq_dist in vocabulario_por_emocion.items()}

# Imprimir los tokens más comunes por emoción
for emotion, tokens in tokens_mas_comunes_por_emocion.items():
  print(f"Tokens más comunes para la emoción '{emotion}':")
  for token, count in tokens:
    print(f"\t{token}: {count}")

In [49]:
# Guardar los tokens más comunes en total
tokens_mas_comunes = []
for emotion in list(df['emotions'].unique()):
  tokens_mas_comunes += list([i[0] for i in FreqDist(vocabulario[emotion]).most_common(8300)])

In [ ]:
tokens_mas_comunes = set(tokens_mas_comunes)
len(tokens_mas_comunes)

In [73]:
# Se guardan los tokens más comunes
with open("most_common_tokens.pkl", "wb") as file:
  pickle.dump(tokens_mas_comunes, file)

In [52]:
# Funcion para limpiar y asignar los tokens en el dataset
def obtener_tokens_de_entrenamiento(text: str) -> str:

  """
  Función para tokenizar y limpiar el texto.
  Se obvian los tokens que sean stopwords,
  nombres, números, caracteres especiales,
  letras y palabras indeseadas. Los demás tokens
  se lematizan y son devueltos unidos en un string
  """

  tokens_crudos = word_tokenize(text)
  tokens = []
  for token in tokens_crudos:
    if token in stopwords: continue
    if token in nombres: continue
    if not token.isalpha():  continue
    if (len(token) < 2): continue
    token = token.lower()
    if token in palabras_indeseadas: continue
    if token not in tokens_mas_comunes: continue
    token = lematizador.lemmatize(token)
    tokens.append(token)

  return ' '.join(tokens)

In [53]:
# Agregamos los tokens correspondiente a cada review
df['Tokens'] = df["text"].apply(lambda x: obtener_tokens_de_entrenamiento(x))

In [ ]:
df.head()

In [55]:
# Creando el training_df (legacy)

training_df = pd.concat([df[df["emotions"] == emotion].sample(6000, replace=True, random_state=1) for emotion in df["emotions"].unique()])

In [ ]:
# Creando el training_df (actual)

# Crear un diccionario con la cantidad de muestras deseadas para cada emoción

desired_samples = {
    'sadness': 9500,
    'appreciation': 10500,
    'gratitude': 8000,
    'disapproval': 9500,
    'joy': 9500,
    'disappointment': 9000,
    'realization': 6000,
    'perplexity': 8000,
    'irritation': 11500,
    'compassion': 8000,
    'embarrassment': 4000,
    'surprise': 4000,
    'pride': 2000,
    'desire': 4000,
    'relief': 2000,
    'fear': 4000,
    'nervousness': 2500,
    'disgust': 6000,

}

# Crear un DataFrame vacío para almacenar las muestras
training_df = pd.DataFrame()

# Iterar sobre las emociones y obtener las muestras deseadas
for emotion, num_samples in desired_samples.items():
    # Obtener las muestras para la emoción actual
    emotion_samples = df[df['emotions'] == emotion].sample(num_samples, replace=True, random_state=1)
    # Concatenar las muestras al DataFrame de entrenamiento
    training_df = pd.concat([training_df, emotion_samples])

# Imprimir el DataFrame de entrenamiento
print(training_df)


In [ ]:
len(training_df)

In [ ]:
training_df.sample(10)

In [59]:
# Vectorizamos la data para entrarla al modelo
vectorizer = TfidfVectorizer(vocabulary=tokens_mas_comunes)
x = vectorizer.fit_transform(training_df["Tokens"])

In [72]:
with open("vectorizador.pkl", "wb") as file:
  pickle.dump(x, file)

In [61]:
# Asignamos las etiquetas
y = training_df["emotions"]

In [62]:
# Dividimos la data que utilizaremos
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=0)

In [63]:
# from sklearn.linear_model import LogisticRegression = dura mucho
# from sklearn.naive_bayes import MultinomialNB = 0.42
# from sklearn.svm import LinearSVC = 0.47
# from sklearn.ensemble import RandomForestClassifier = 0.55 (buen modelo)
# from sklearn.ensemble import GradientBoostingClassifier
# from sklearn.ensemble import ExtraTreesClassifier = 0.55 (bueno, pero randomForest mejor)
# from sklearn.ensemble import VotingClassifier
# from sklearn.ensemble import AdaBoostClassifier = Malo
# from sklearn.ensemble import BaggingClassifier
# from xgboost import XGBClassifier

# Modelo más efectivo en primera instancia
random_forest_model = RandomForestClassifier(random_state=0)

# Lista de modelos
models_list = [random_forest_model]

In [64]:
def entrenamiento_de_modelos(lista_de_modelos: list, X_train, y_train, X_test, y_test):

  """
  Entrena la lista de modelos que le pasen
  con la data vectorizada con tfidf y se
  recolectan las métricas
  """
  lista_de_resultados = []

  for modelo in lista_de_modelos:
    # Entrenamiento
    modelo.fit(X_train.toarray(), y_train)
    y_prediccion = modelo.predict(X_test.toarray())

    # Métricas
    report = classification_report(y_prediccion, y_test)
    matrix = confusion_matrix(y_prediccion, y_test)
    accuracy = accuracy_score(y_prediccion, y_test)

    # Añadiendo los reusltados
    lista_de_resultados.append((str(modelo), report, matrix, accuracy))

  return lista_de_resultados


In [65]:
# Se entrenan los modelos y se obtienen las analíticas
resultados_de_modelos = entrenamiento_de_modelos(models_list, X_train, y_train, X_test, y_test)

In [ ]:
# Analisis de los modelos
for resultado in resultados_de_modelos:
  print(f"""
  {resultado[0]}

  {resultado[1]}

  {resultado[2]}

  {resultado[3]}

  ------------------------------------------------------------ """)

In [74]:
final_emotion_model = random_forest_model.fit(X_train.toarray(), y_train)

In [75]:
# Guardamos el mejor modelo
with open("emotion_model.pkl", "wb") as file:
  pickle.dump(final_emotion_model, file)